In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import auc,roc_curve, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix,classification_report,f1_score
from scipy.stats import norm
from sklearn import preprocessing
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
# from tensorflow.keras.layers.convolutional import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop,Adam

import tensorflow.keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
from functools import partial
import random
random_seed = 42
MAX_INT = np.iinfo(np.int32).max
from tensorflow.keras.layers import LeakyReLU

In [2]:
def deviation_loss(y_true, y_pred):
    '''
    z-score-based deviation loss
    '''    
    confidence_margin = 5.0     ### 修改了这里
    inlier_loss = K.abs(y_pred) 
    outlier_loss = K.maximum(confidence_margin - y_pred, 0.)
    return (1 - y_true) * inlier_loss + y_true * outlier_loss

In [3]:
def network(input_shape):
    '''
    network architecture with one hidden layer
    '''
    x_input = Input(shape=input_shape)
    intermediate = Dense(20, activation='relu', 
                kernel_regularizer=regularizers.l2(0.01), name = 'hl1')(x_input)## 修改了这里
    intermediate = Dense(1, activation='linear', name = 'score')(intermediate)    
    return Model(x_input, intermediate)

In [4]:
def anomaly_score_learner(input_shape, lr):
    '''
    construct the deviation network-based detection model
    '''
    model = network(input_shape)
    rms = RMSprop(clipnorm = 1.,learning_rate = lr) ### 修改了这里
    model.compile(loss = deviation_loss, optimizer = rms)
    return model

In [5]:
def load_model_weight_predict(model_name, input_shape, x_test):
    '''
    load the saved weights to make predictions
    '''
    model = deviation_network(input_shape)
    model.load_weights(model_name)
    scoring_network = Model(inputs=model.input, outputs=model.output)    
    
    scores = scoring_network.predict(x_test)
    return scores

In [6]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


Test_data = pd.read_csv('data/celeba/x_test.csv',index_col= 0)
Train_data = pd.read_csv('data/celeba/x_train.csv',index_col= 0)

print('训练集大小为{}'.format(Train_data.shape))
print(Counter(Train_data.loc[:,'class']))


print('测试集大小为{}'.format(Test_data.shape))
print(Counter(Test_data.loc[:,'class']))

训练集大小为(162079, 40)
Counter({0: 158441, 1: 3638})
测试集大小为(40520, 40)
Counter({0: 39611, 1: 909})


In [11]:
x_train_normal = Train_data[(Train_data['class'] == 0)].reset_index(drop=True).iloc[:,:-1].values
x_train_normal = x_train_normal.astype(np.float32)
x_train_normal[np.isnan(x_train_normal)] = 0
print(x_train_normal.shape)


x_train_vandal = Train_data[(Train_data['class'] == 1)].sample(n=70,random_state=42).reset_index(drop=True).iloc[:,:-1].values
x_train_vandal = x_train_vandal.astype(np.float32)
x_train_vandal[np.isnan(x_train_vandal)] = 0
print(x_train_vandal.shape)


x_test = Test_data.iloc[:,:-1].values
x_test = x_test.astype(np.float32)
x_test[np.isnan(x_test)] = 0
y_test = Test_data.iloc[:,-1].values
Counter(y_test)

(158441, 39)
(7, 39)


Counter({0: 39611, 1: 909})

In [15]:
def sample_shuffle_uspv(X, labels, weights):
#     print(X.shape,labels.shape,weights.shape)
    n_samples = len(X)
    s = np.arange(n_samples)
    np.random.shuffle(s)
    return np.array(X[s]), labels[s], weights[s]

In [16]:
import random
# Dataset iterator
def inf_normal_train_gen(BATCH_SIZE,normal_data_):
    n_samples = len(normal_data_)
    s = np.arange(n_samples)
    while True:
        np.random.shuffle(s)
        normal_data_ = np.array(normal_data_[s])
        for i in range(int(len(normal_data_)/BATCH_SIZE)):
            yield normal_data_[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
            
# Dataset iterator
def inf_vandal_train_gen(BATCH_SIZE,vandal_data_):
    n_samples = len(vandal_data_)
    s = np.arange(n_samples)
    while True:
        np.random.shuffle(s)
        vandal_data_ = np.array(vandal_data_[s])
        dataset = []
        for i in range(BATCH_SIZE):
            dataset.append(random.choice(vandal_data_))
        dataset = np.array(dataset, dtype='float32')
        yield dataset

In [17]:
def inf_hard_normal_weight_train_gen(BATCH_SIZE,normal_scores,normal_data_):
    indicator = np.sign(np.abs(normal_scores) - 5.0)
    condition = np.greater(indicator, np.zeros_like(indicator))
    mask_tar = np.where(condition, np.zeros_like(indicator),np.tanh(np.abs(normal_scores)))
    dataset = []
    weights = []
#     mean_scores = np.mean(mask_tar)
    mean_scores = 0
    candicate_idxs = np.where(mask_tar>mean_scores)[0]
    while True:
        for idx in candicate_idxs:
            tar = random.uniform(mean_scores, 1.0) ### 修改了这里
            if mask_tar[idx]>tar:
                dataset.append(normal_data_[idx])
                weights.append(np.abs(normal_scores[idx]))
            if len(dataset)==BATCH_SIZE:
                dataset = np.array(dataset, dtype='float32')
                weights = np.array(weights, dtype='float32')
#                 print(idx)
                yield dataset,weights
                dataset = []
                weights = []

In [18]:
class WGANGP():
    def __init__(self, input_dim):
        self.latent_dim = 32
        self.input_dim = input_dim
        self.n_critic = 5
        self.batch_size = 256
        optimizer = Adam(0.0001, beta_1=0.5, beta_2=0.9)
        self.generator = self.build_generator()
        self.critic = self.build_critic()
        
        self.generator.trainable = False
        
        real_data = Input(shape = (self.input_dim,))
        z_disc = Input(shape = (self.latent_dim,))
        hard_weight = Input(shape = (1,))
        
        fake_data = self.generator(z_disc)
        
        disc_real = self.critic(real_data) * hard_weight
        disc_fake = self.critic(fake_data) * hard_weight
        
        alpha = K.random_uniform([self.batch_size,1], 0, 1)
        interpolates = alpha * real_data + (1 - alpha) * fake_data
        disc_interpolates = self.critic(interpolates)
        partial_gp_loss = partial(self.gradient_penalty_loss,averaged_samples=interpolates,hard_weight_=hard_weight)
        partial_gp_loss.__name__ = 'gradient_penalty'
        
        self.critic_model = Model(inputs=[real_data,z_disc,hard_weight],
                                  outputs=[disc_real,disc_fake,disc_interpolates])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                        self.wasserstein_loss,
                                        partial_gp_loss],optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        
        self.critic.trainable = False
        self.generator.trainable = True
        
        z_gen = Input(shape=(self.latent_dim,))
        generate_data = self.generator(z_gen)
        disc_generate = self.critic(generate_data)
        self.generator_model = Model(z_gen,disc_generate)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)
        
    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples,hard_weight_):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(hard_weight_ * gradient_penalty)
    
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)
    
    def build_generator(self):
        model = Sequential()
        model.add(Dense(512,activation="relu",input_dim=self.latent_dim))
        model.add(Dense(512,activation="relu"))
        model.add(Dense(512,activation="relu"))
        model.add(Dense(input_dim))
        model.add(LeakyReLU())
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        output = model(noise)
        
        return Model(noise,output)
    
    def build_critic(self):
        model = Sequential()
        model.add(Dense(128,activation="relu",input_dim=self.input_dim))
        model.add(Dense(64,activation="relu"))
        model.add(Dense(1,activation="linear"))
        model.summary()
        
        inputs = Input(shape=(self.input_dim,))
        output = model(inputs)
        
        return Model(inputs,output)
    
    def train(self, epochs, batch_size, weight_model,x):
        x_scores = weight_model.predict(x)
        gen = inf_hard_normal_weight_train_gen(batch_size, x_scores, x)
        real_label = -np.ones((batch_size,1))
        fake_label = np.ones((batch_size,1))
        dummy = np.zeros((batch_size,1))
        for epoch in range(epochs):
            for _ in range(self.n_critic):
                _data,_weight = gen.__next__()
                _noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                disc_loss = self.critic_model.train_on_batch([_data, _noise, _weight],
                                                                [real_label, fake_label, dummy])
            
            gen_loss = self.generator_model.train_on_batch(_noise, real_label)
        print(disc_loss, gen_loss)
#         print ("%d [D loss: %f] [G loss: %f]" % (epoch, disc_loss[0], gen_loss))
        return disc_loss[0]

In [19]:
input_shape = x_train_normal.shape[1:]
epoch_learner = 30
epoch_gen = 1000
batch_size = 128
nb_batch = int(len(x_train_normal)/batch_size)
lr = 0.0001
required_improvements = 2 ## early stopping
saved_filpath = 'saved_model.h5'

In [1]:
random_seed = 42

for run in np.arange(1):
    rng = np.random.RandomState(random_seed)
    
    model = anomaly_score_learner(input_shape, lr)
    wgan = WGANGP(input_shape)
    
    gen_normal = inf_normal_train_gen(batch_size,x_train_normal)
    gen_vandal = inf_vandal_train_gen(batch_size,x_train_vandal)
    
    best_cost = np.iinfo(np.int32).max
    
    for i in range(epoch_learner):
        
        print(str(i)+'*'*20)
        if i!=0:
            disc_loss = np.abs(wgan.train(epochs = epoch_gen, batch_size = 256, weight_model = model, x = x_train_normal))
            generator_quality = np.exp(- disc_loss)            
            
        avg_cost = 0  
        for j in range(nb_batch):
            normal_data_batch = gen_normal.__next__()
            normal_data_label = np.zeros((batch_size,1))
            normal_data_weight = np.ones(batch_size)
            vandal_data_batch = gen_vandal.__next__()
            vandal_data_label = np.ones((batch_size,1))
            vandal_data_weight = np.ones(batch_size)
            
            if i == 0: ###修改了这里
                data_batch = np.vstack((normal_data_batch,vandal_data_batch))
                data_label = np.vstack((normal_data_label,vandal_data_label))
                data_weight = np.hstack((normal_data_weight,vandal_data_weight))
                
            else:
                _noise = np.random.normal(0, 1, (int(batch_size), 32))
                fake_data_batch = wgan.generator.predict(_noise)
                fake_data_label = np.zeros((int(batch_size),1))
                fake_data_weight =  generator_quality * np.ones(int(batch_size))
                data_batch = np.vstack((normal_data_batch,vandal_data_batch,fake_data_batch))
                data_label = np.vstack((normal_data_label,vandal_data_label,fake_data_label))
                data_weight = np.hstack((normal_data_weight,vandal_data_weight,fake_data_weight))
            
            data_batch, data_label,data_weight = sample_shuffle_uspv(data_batch,data_label,data_weight)
            #给样本加入权重
            d_loss_train = model.train_on_batch(data_batch, data_label,sample_weight = data_weight)
            
            avg_cost += d_loss_train
        
        print("*****测试集********")
        scores = model.predict(x_test)
        precision, recall, threshold = precision_recall_curve(y_test, scores)
        pr_auc = auc(recall, precision) 
        print(roc_auc_score(y_test,scores),pr_auc)
        
        
        if avg_cost < best_cost:
                model.save(saved_filpath)
                res = [roc_auc_score(y_test,scores),pr_auc]
                best_cost = avg_cost
                last_improvements = 0
            else:
                last_improvements += 1 

            if last_improvements >= required_improvements:
                break;
    
    print("final_result:" + res)